In [33]:

from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pandas as pd
import requests
from us import states

STATES = [str(s) for s in states.STATES]
SPECIAL_CASES = {"UConn": "University of Connecticut", "Illinois": "University of Illinois Urbana-Champaign",
                "Texas": "University of Texas Austin"}

In [36]:
from team import dataframe
from IPython.display import display

df = dataframe.get_all_d1_schools()
dataframe.add_ap_rankings_to_dataframe(df)
display(df[df["School"] == "Gonzage University"])


,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking


In [29]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_men%27s_basketball_programs").content

In [30]:
souper = BeautifulSoup(data, "html.parser")
table = souper.find("table")
columns = [c.string.strip() for c in table.find_all("th")]
columns.append("AP Ranking")
all_school_data = list()
for school in table.find_all("tr")[1:]:
    school_data = list()
    for d in school.find_all("td"):
        content = d.string
        if not content:
            content = d.find("a").text
        content = content.strip()
        school_data.append(content)
    school_data.append(None)
    all_school_data.append(school_data)
df = pd.DataFrame(all_school_data, columns = columns)
display(df)

,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking
0,"University at Albany, SUNY",Great Danes,SEFCU Arena,America East,"(5) 2006, 2007, 2013, 2014, 2015",(0),(0),None
1,Binghamton University,Bearcats,Binghamton University Events Center,America East,(1) 2009,(0),(0),None
2,University of Hartford,Hawks,Chase Arena at Reich Family Pavilion,America East,(1) 2021,(0),(0),None
3,University of Maine,Black Bears,Alfond Arena,America East,(0),(0),(0),None
4,"University of Maryland, Baltimore County",Retrievers,Retriever Activities Center,America East,"(2) 2008, 2018",(0),(0),None
...,...,...,...,...,...,...,...,...
353,Seattle University,Redhawks,KeyArena,WAC,"(11) 1953, 1954, 1955, 1956, 1958, 1961, 1962,...",(1) 1958,(0),None
354,Stephen F. Austin State University,Lumberjacks,William R. Johnson Coliseum,WAC,"(5) 2009, 2014, 2015, 2016, 2018",(0),(0),None
355,Tarleton State University,Texans,Wisdom Gymnasium,WAC,(0),(0),(0),None
356,Utah Valley University,Wolverines,UCCU Center,WAC,(0),(0),(0),None


In [31]:
ap_ranks_raw_data = requests.get("https://www.ncaa.com/rankings/basketball-men/d1/associated-press").content

In [32]:
ap_ranks = BeautifulSoup(ap_ranks_raw_data, "html.parser")
for d in ap_ranks.find("table").find_all("tr")[1:]:
    td = d.find_all("td")
    best_ratio = 0
    best_name = None
    best_index = -1
    for i, row in df.iterrows():
        ap_name = td[1].string
        if ap_name[-1] == ")" and ap_name[-4] == "(":
            ap_name = ap_name[:-5]
        elif ap_name[-1] == ")" and ap_name[-3] == "(":
            ap_name = ap_name[:-4]
        if ap_name in SPECIAL_CASES:
            ap_name = SPECIAL_CASES[ap_name]
        elif ap_name in STATES:
            ap_name = f"University of {ap_name}"
        ratio = fuzz.ratio(row["School"], ap_name)
        if ratio > best_ratio:
            best_ratio = ratio
            best_name = row["School"]
            best_index = i
    df.at[best_index, "AP Ranking"] = int(td[0].string)